### Comparing Aggregate Models for Regression

This try-it focuses on utilizing ensemble models in a regression setting.  Much like you have used individual classification estimators to form an ensemble of estimators -- here your goal is to explore ensembles for regression models.  As with your earlier assignment, you will use scikitlearn to carry out the ensembles using the `VotingRegressor`.   


#### Dataset and Task

Below, a dataset containing census information on individuals and their hourly wage is loaded using the `fetch_openml` function.  OpenML is another repository for datasets [here](https://www.openml.org/).  Your task is to use ensemble methods to explore predicting the `wage` column of the data.  Your ensemble should at the very least consider the following models:

- `LinearRegression` -- perhaps you even want the `TransformedTargetRegressor` here.
- `KNeighborsRegressor`
- `DecisionTreeRegressor`
- `Ridge`
- `SVR`

Tune the `VotingRegressor` to try to optimize the prediction performance and determine if the wisdom of the crowd performed better in this setting than any of the individual models themselves.  Report back on your findings and discuss the interpretability of your findings.  Is there a way to determine what features mattered in predicting wages?

In [86]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import shap

In [87]:
data = fetch_openml(data_id=534, as_frame=True).frame

The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.


In [88]:
data.head()

,EDUCATION,SOUTH,SEX,EXPERIENCE,UNION,WAGE,AGE,RACE,OCCUPATION,SECTOR,MARR
0,8,no,female,21,not_member,5.10,35,Hispanic,Other,Manufacturing,Married
1,9,no,female,42,not_member,4.95,57,White,Other,Manufacturing,Married
2,12,no,male,1,not_member,6.67,19,White,Other,Manufacturing,Unmarried
3,12,no,male,4,not_member,4.00,22,White,Other,Other,Unmarried
4,12,no,male,17,not_member,7.50,35,White,Other,Other,Married


In [98]:
# Predict Wage
X = data.drop(columns=['WAGE'])
y = data['WAGE']

In [99]:
# Categorical and numerical feature split
categorical_features = ['SOUTH', 'SEX', 'UNION', 'RACE', 'OCCUPATION', 'SECTOR', 'MARR']
numerical_features = ['EDUCATION', 'EXPERIENCE', 'AGE']


In [100]:
# Preprocessing pipeline for categorical (one-hot) and numeric (scaling) features
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [92]:
# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [93]:
# Model pipelines
models = {
    'LinearRegression': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', LinearRegression())]),
    'Ridge': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', Ridge())]),
    'KNeighborsRegressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', KNeighborsRegressor())]),
    'DecisionTreeRegressor': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', DecisionTreeRegressor())]),
    'SVR': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', SVR())])
}

In [94]:
# Train and evaluate individual models
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print(f'{name}: MSE = {mean_squared_error(y_test, y_pred)}, R^2 = {r2_score(y_test, y_pred)}')


LinearRegression: MSE = 19.502597324112376, R^2 = 0.3882212186709215
Ridge: MSE = 19.504597676303987, R^2 = 0.3881584694377974
KNeighborsRegressor: MSE = 19.771630616822435, R^2 = 0.3797818884004033
DecisionTreeRegressor: MSE = 49.28300490654206, R^2 = -0.5459631443388546
SVR: MSE = 20.983530209733768, R^2 = 0.3417656978529092


In [102]:
# Define preprocessing
categorical_features = ['SOUTH', 'SEX', 'UNION', 'RACE', 'OCCUPATION', 'SECTOR', 'MARR']
numeric_features = ['EDUCATION', 'EXPERIENCE', 'AGE']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)])

# Models
models = [
    ('lr', LinearRegression()),
    ('knn', KNeighborsRegressor()),
    ('tree', DecisionTreeRegressor()),
    ('ridge', Ridge()),
    ('svr', SVR())
]
# Voting regressor with equal weights for simplicity
voting_regressor = VotingRegressor(estimators=models)

# Pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('voting_regressor', voting_regressor)])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model training
pipeline.fit(X_train, y_train)

# Predictions
y_pred = pipeline.predict(X_test)

# Evaluation
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'R^2: {r2}')


MSE: 21.236535301774214
R^2: 0.33382915769334875


In [103]:
# Permutation Importance
result = permutation_importance(pipeline, X_test, y_test, n_repeats=10, random_state=42)
importance_scores = result.importances_mean

# Report feature importance
print("Permutation Importance Scores:")
for feature, score in zip(X.columns, importance_scores):
    print(f"{feature}: {score}")

Permutation Importance Scores:
EDUCATION: 0.334338738425728
SOUTH: -0.0005318648708762419
SEX: 0.07163808869393483
EXPERIENCE: 0.021027444150094464
UNION: 0.003359527041061572
AGE: 0.023951297010808413
RACE: 0.003951351387968982
OCCUPATION: 0.09332833780412698
SECTOR: 0.004527812316646162
MARR: -0.004540294897846764


In [80]:
# Tuning Voting Regressor (ensemble)

from sklearn.model_selection import GridSearchCV

# Define the base models with pipelines
models = {
    'lr': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', LinearRegression())]),
    'ridge': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', Ridge())]),
    'knn': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', KNeighborsRegressor())]),
    'tree': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', DecisionTreeRegressor())]),
    'svr': Pipeline(steps=[('preprocessor', preprocessor), ('regressor', SVR())])
}


# Create the Voting Regressor without weights (initial version)
voting_regressor = VotingRegressor(
    estimators=[
        ('lr', models['lr']),
        ('ridge', models['ridge']),
        ('knn', models['knn']),
        ('tree', models['tree']),
        ('svr', models['svr'])
    ]
)

# Define weight combinations to search over
param_grid = {
    'weights': [
        [1, 1, 1, 1, 1],   # Equal weights
        [1, 2, 1, 1, 1],   # More weight to Ridge
        [2, 1, 1, 1, 1],   # More weight to Linear Regression
        [1, 1, 2, 1, 1],   # More weight to KNN
        [1, 1, 1, 2, 1],   # More weight to Decision Tree
        [1, 1, 1, 1, 2],   # More weight to SVR
        [0.5, 2, 0.5, 1, 2], # Different combination of weights
        # Add more combinations as needed
    ]
}

# GridSearchCV to find the best weights
grid_search = GridSearchCV(voting_regressor, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best estimator and weights
best_voting_regressor = grid_search.best_estimator_
best_weights = grid_search.best_params_['weights']

# Predict using the tuned Voting Regressor
y_pred_tuned = best_voting_regressor.predict(X_test)

# Evaluate performance
mse_tuned = mean_squared_error(y_test, y_pred_tuned)
r2_tuned = r2_score(y_test, y_pred_tuned)

print(f'Best Weights: {best_weights}')
print(f'Tuned VotingRegressor: MSE = {mse_tuned}, R^2 = {r2_tuned}')

Best Weights: [2, 1, 1, 1, 1]
Tuned VotingRegressor: MSE = 20.884409156820546, R^2 = 0.3448750353399902


In [82]:
from sklearn.inspection import permutation_importance

X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Calculate Permutation Importance
perm_importance = permutation_importance(best_voting_regressor, X_test_transformed, y_test, n_repeats=10, random_state=42)

# Display Permutation Importance
importances = perm_importance.importances_mean
feature_names = preprocessor.get_feature_names_out()
importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
importance_df = importance_df.sort_values(by='importance', ascending=False)

print(importance_df)

ValueError: X has 23 features, but ColumnTransformer is expecting 10 features as input.